In [113]:
import numpy as np

from macd_strategy import MACDandEMA

from quantfreedom.enums import DynamicOrderSettingsArrays
from quantfreedom.enums import *
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.simulate import run_df_backtest, or_backtest


np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [114]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol='BTCUSDT',
    timeframe='1h',
    candles_to_dl=3000
    )

In [115]:
macd_strat = MACDandEMA(
    long_short= "long",
    ema_length=np.arange(100, 501, 100),
    fast_length=np.arange(10, 61, 10),
    macd_below=np.arange(0, 51, 25),
    signal_smoothing=np.arange(5, 16, 5),
    slow_length=np.arange(30, 121, 30),
)

In [116]:
backtest_settings = BacktestSettings()
    
exchange_settings = ExchangeSettings(
    asset_tick_step=3,             # Decimal precision for Bitcoin can be high since it can be priced very granularly
    leverage_mode=1,  # Isolated leverage is safer for managing risk in volatile markets
    leverage_tick_step=2,       # Allows for fine control over leverage adjustments
    limit_fee_pct=0.0003,          # Typical limit order fee percentage for lower volume tiers on Kraken
    market_fee_pct=0.0006,         # Typical market order fee percentage for lower volume tiers on Kraken
    max_asset_size=100.0,            # Practical upper limit for the size of a single Bitcoin order
    max_leverage=150.0,              # Maximum allowable leverage for Bitcoin; check current regulatory limits
    min_asset_size=0.001,          # Minimum trade size for Bitcoin on Kraken
    min_leverage=1.0,              # Minimum leverage available (effectively no leverage)
    mmr_pct=0.004,                  # Maintenance margin requirement at 4% is typical for Bitcoin
    position_mode=3,               # Assuming '1' is for netted positions, which is typical for individual traders
    price_tick_step=1,            # Minimum price movement for Bitcoin, appropriate given its value
)

In [117]:
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,    # Increase position size based on risk percentage of account size
    leverage_strategy_type=LeverageStrategyType.Dynamic,    # Leverage is dynamically adjusted based on account size
    pg_min_max_sl_bcb="min",    # Stop loss is based on the minimum of the last two candle bodies
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,  # Stop loss is based on the candle body
    sl_to_be_bool=False,    # Stop loss is not based on the candle body
    starting_bar=50,    # Start the strategy after 50 bars
    starting_equity=1000.0,   # Start with $1000
    static_leverage=None,   # Leverage is dynamically adjusted based on account size
    tp_fee_type="limit",    # Take profit is based on the limit fee
    tp_strategy_type=TakeProfitStrategyType.RiskReward,   # Take profit is based on the risk-reward ratio
    trail_sl_bool=False,    # Trailing stop loss is not used
    z_or_e_type=None,   # Z-score or entropy is not used
)

In [118]:
dos_arrays = DynamicOrderSettingsArrays(
    account_pct_risk_per_trade=np.array([12]),
    max_trades=np.array([0]),
    risk_reward=np.array([2, 5]),
    sl_based_on_add_pct=np.array([0.1, 0.25, 0.5]),
    sl_based_on_lookback=np.array([20, 50]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5, 1.0]),
    trail_sl_when_pct=np.array([1, 2]),
)

In [119]:
backtest_results = run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=macd_strat,
)

TypeError: 'NoneType' object is not subscriptable